## Exercice 5

In [551]:
import pandas as pd
from requests import get
import requests as rq

In [552]:
df = pd.read_csv("https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv", sep = ";")

In [553]:
df.describe()

,username,ip_address,product,price,infos
count,200,200,200,200,200
unique,200,200,188,200,133
top,etremain2d,160.32.117.250,Squash - Guords,756.85,Ingredients: sugar
freq,1,1,2,1,7


#### Traitement de la colonne infos

In [554]:
df["infos"] = df["infos"].str.lower().str.replace(',', '').str.split()

In [555]:
words = df.infos.apply(lambda x: pd.value_counts(x)).sum(axis=0)

Dans la liste de mots, nous pouvons voir que certains sont des ingrédients, d'autres non. Nous allons supprimer les mots qui ne nous intéressent pas.

In [556]:
words

contain          63.0
sugar           150.0
may              63.0
fish             84.0
contains         60.0
peanut           45.0
and             169.0
ingredients:     77.0
mustard          40.0
gluten           55.0
egg              47.0
milk             47.0
soja             56.0
dtype: float64

In [557]:
list = ['may', 'contain', 'contains', 'and', 'ingredients:', 'gluten']
df['infos'] = df['infos'].apply(
    lambda x: [item for item in x if item not in list])

In [558]:
dummies = pd.get_dummies(df['infos'].apply(pd.Series).stack()).sum(level=0)

In [559]:
df = df.merge(dummies, left_index=True, right_index=True)
df = df.drop("infos", axis=1)
df.head()

,username,ip_address,product,price,egg,fish,milk,mustard,peanut,soja,sugar
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,0,0,0,0,0,0,1
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,0,1,0,0,1,0,0
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,0,1,0,1,0,0,0
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,1,1,0,0,0,0,1
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33 MGA,0,0,1,0,0,0,1


#### Traitement de la colonne price

In [560]:
new = df["price"].str.split(" ", n = 1, expand = True) 

In [561]:
df["price"] = new[0]
df["currency"] = new[1]

In [562]:
df.head()

,username,ip_address,product,price,egg,fish,milk,mustard,peanut,soja,sugar,currency
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,0,0,0,0,0,0,1,None
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,0,1,0,0,1,0,0,None
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,0,1,0,1,0,0,0,None
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,1,1,0,0,0,0,1,None
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,0,0,1,0,0,0,1,MGA


In [563]:
def f(row):
    if row['currency'] == None:
        url = 'https://ipapi.co/' + row['ip_address'] + '/currency/'
        val = get(url).text
    else:
        val = row['currency']
    return val

In [564]:
df['estimated_currency'] = df.apply(f, axis = 1)

In [565]:
url = 'https://api.exchangerate-api.com/v4/latest/EUR'
response = rq.get(url)
data = response.json()
table_rates = data['rates']

In [566]:
def g(row):
    if row['estimated_currency'] == 'Undefined':
        val = 'None'
    else:
        curr = row['estimated_currency']
        val = 'None'
        if curr in table_rates.keys():
            rates_chg = table_rates[curr]
            val = float(row['price']) / float(rates_chg)
    return val

In [567]:
df['price_eur'] = df.apply(g, axis=1)

In [568]:
df = df.drop(["currency","estimated_currency"], axis=1)

In [569]:
df.head()

,username,ip_address,product,price,egg,fish,milk,mustard,peanut,soja,sugar,price_eur
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,0,0,0,0,0,0,1,None
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,0,1,0,0,1,0,0,None
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,0,1,0,1,0,0,0,None
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,1,1,0,0,0,0,1,7.8543
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,0,0,1,0,0,0,1,None
